# Integrating human and macaque caudate snATAC-seq

Notes on linked peaks notes: 
- human peak w/ macaque ortholog intersecting human ortholog of macaque peak
- call ties w/ multiple peak matches using nearest summits
- count matrix comes from linked peaks 1-1 in human and macaque 

Notes on integration:
- perform integration with about same number of cells in each group
- here selected representative human and macaque subject
- predefine the features during anchor finding step

In [1]:
PROJDIR=file.path('../../../data/raw_data/primate_peak_orthologs')

#######################################
### set up libraries and functions ####
ss <- function(x, pattern, slot = 1, ...) { 
  sapply(strsplit(x = x, split = pattern, ...), '[', slot) }
options(stringsAsFactors = F, repr.plot.width=14, repr.plot.height=6)
suppressMessages(library(Signac)); suppressMessages(library(Seurat))
suppressMessages(library(harmony))

source('../hal_scripts/narrowPeakFunctions.R')

In [2]:
# # set up future for parallelization
library(future)
library(future.apply)
plan("sequential")
options(future.globals.maxSize = 180 * 1024^3)

# 1) prepare integrated human and 

In [3]:
integratedRDS_fn = file.path(PROJDIR,'rdas','linkOrthologSeuratCCAHumanOnly.rds')
human = readRDS(file = integratedRDS_fn)

Read in the macaque snATAC-seq files.

In [4]:
## load the seurat object
saveRDS_fn = file.path(PROJDIR, 'rdas', 'linkOrthologMatrixSeurat.rds')
obj_seurat = readRDS(file = saveRDS_fn)

### grab just the macaque cells ###
cells = WhichCells(obj_seurat, expression = Species %in% c("rheMac10"))
macaque = subset(obj_seurat, cells = cells)
rm(obj_seurat); gc(verbose = FALSE)

## show cell clusters per sample
macaque@meta.data$log10nFrags = log10(macaque@meta.data$nFrags)
table(macaque@meta.data$Clusters2, macaque@meta.data$Sample)

## recompute TFIDR and SVD on just macaque samples
macaque <- RunTFIDF(macaque, verbose = FALSE)
macaque <- RunSVD(macaque, verbose = FALSE)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,11016207,588.4,17062155,911.3,12508236,668.1
Vcells,2432973957,18562.2,3967120162,30266.8,2891570154,22061.0


             
              CAUD_WS1H_STA682A131
  Astro                       1035
  Interneuron                  193
  Microglia                    519
  MSN_D1                       975
  MSN_D2                       925
  MSN_UNK1                     400
  Oligo                       1442
  OPC                          304

# 2) integration with Seurat CCA

In [6]:
# features <- SelectIntegrationFeatures(object.list = obj_seurat.list, nfeatures = 10000)
features <- rownames(macaque)

# find integration anchors between species, using all features
anchors <- FindIntegrationAnchors(
        object.list = list(human, macaque), reduction = 'cca', anchor.features = features,
        reference = c(1), k.filter = NA, assay = c('integrated', 'peaks'))

Scaling features for provided objects

Finding anchors between all query and reference datasets

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 21789 anchors



In [8]:
# integrate data and create a new merged object
integrated <- IntegrateData(anchors, dims = 2:30, preserve.order = TRUE)

# we now have a "corrected" TF-IDF matrix, and can run LSI again on this corrected matrix
integrated <- RunSVD(integrated, n = 30, reduction.name = 'integratedLSI', verbose = FALSE)
integrated <- RunUMAP(integrated, dims = 2:30, reduction = 'integratedLSI', verbose = FALSE)

ERROR: Error in merged.scaledata[[i]] <- assay.scale[feat.use, ]: more elements supplied than there are to replace


In [ ]:
# plot embeddings
p_seuratIntegration_species = 
    DimPlot(object = integrated, label = FALSE, group.by = 'Species', cols = 'Dark2') +
    ggplot2::ggtitle('Seurat CCA Integration')

p_seuratIntegration_clusters2 = 
    DimPlot(object = integrated, label = TRUE, group.by = 'Clusters2', cols = 'Paired') +
    ggplot2::ggtitle('Seurat CCA Integration')

p_seuratIntegration_species + p_seuratIntegration_clusters2

In [ ]:
DimPlot(object = integrated, label = TRUE, group.by = 'Clusters2', cols = 'Paired',split.by = 'Sample') +
    ggplot2::ggtitle('Seurat CCA Integration')

In [ ]:
integrated
object.size(integrated) / 1024^3

In [ ]:
integrated.trim = DietSeurat(
    integrated, counts = TRUE, data = TRUE, scale.data = FALSE, 
    assays = c('peaks', 'integrated'), dimreducs = c('integratedLSI','umap'))

object.size(integrated.trim) / 1024^3

In [ ]:
## save the seurat object
integratedRDS_fn = file.path(PROJDIR,'rdas','linkOrthologSeuratCCAprimate.rds')
saveRDS(integrated.trim, file = integratedRDS_fn)